The **Continuous Bag-of-Words model (CBOW)** is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

In this notebook, I am going to use `tensorflow 2.0.0-beta0` and take the advantage of its flexible custom model definitions to create a COBW model and will train it to get the embeddings of a toy dataset. 

In [122]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
import numpy as np
from tqdm import tqdm

print(tf.__version__)

2.0.0-beta0


In [3]:
# 2 words to the left, 2 to the right
CONTEXT_SIZE = 2  
# Embedding dimension on left
EMBED_LEFT = 10
# Embedding dimension on right
EMBED_RIGHT = 10

In [2]:
# Row test from Shakespeare Sonnet 2
raw_text = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['When', 'forty', 'shall', 'besiege'], 'winters'), (['forty', 'winters', 'besiege', 'thy'], 'shall'), (['winters', 'shall', 'thy', 'brow,'], 'besiege'), (['shall', 'besiege', 'brow,', 'And'], 'thy'), (['besiege', 'thy', 'And', 'dig'], 'brow,')]


The toy dataset is ready. Let's now specify the model definition. 

In [158]:
class CBOW(Model):

    def __init__(self, vocab_size, embedding_dim_left, embedding_dim_right, context_size):
        super(CBOW, self).__init__()
        self.embeddings_left = Embedding(vocab_size, embedding_dim_left)
        self.embeddings_right = Embedding(vocab_size, embedding_dim_right)
        self.linear1 = Dense(128, activation='linear', input_dim=(context_size * embedding_dim_left))
        self.linear2 = Dense(vocab_size, activation='linear', input_dim=128)

    def call(self, inputs):
        left_embeds = tf.reshape(self.embeddings_left(inputs[:2]), [1,20])
        right_embeds = tf.reshape(self.embeddings_left(inputs[:2]), [1,20])
        left_out = tf.keras.activations.relu(self.linear1(left_embeds))
        right_out = tf.keras.activations.relu(self.linear1(right_embeds))
        out = self.linear2(left_out + right_out)
        return out

Let's instantiate the model. 

In [175]:
model = CBOW(vocab_size, EMBED_LEFT, EMBED_RIGHT, CONTEXT_SIZE)

Define our loss function and optimizer. 

In [176]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

The function which will train our model. We use `GradientTape` to make use of automatic differentiation. Makes the process of Backpropagation much more flexible. Makes it also easy to chart dependencies inside the training loop. 

In [177]:
@tf.function
def model_train(features, labels):
    # Define the GradientTape context
    with tf.GradientTape() as tape:
        # Get the probabilities
        predictions = model(features)
        # Calculate the loss
        loss = loss_func(labels, predictions)
    # Get the gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    # Update the weights
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

Finally we train the model for 10 epochs. 

In [178]:
running_loss = []
for epoch in tqdm(range(10)):
    total_loss = 0
    for context, target in data:
        context_idxs = np.array([word_to_ix[w] for w in context])
        target_idxs = np.array([word_to_ix[target]])
        loss = model_train(context_idxs, target_idxs)
        total_loss += loss
    running_loss.append(total_loss)
print(running_loss)

100%|██████████| 10/10 [00:01<00:00,  8.50it/s]

[<tf.Tensor: id=66937, shape=(), dtype=float32, numpy=1550.9991>, <tf.Tensor: id=67382, shape=(), dtype=float32, numpy=1634.5785>, <tf.Tensor: id=67827, shape=(), dtype=float32, numpy=1486.1724>, <tf.Tensor: id=68272, shape=(), dtype=float32, numpy=1476.1912>, <tf.Tensor: id=68717, shape=(), dtype=float32, numpy=1470.2799>, <tf.Tensor: id=69162, shape=(), dtype=float32, numpy=1466.611>, <tf.Tensor: id=69607, shape=(), dtype=float32, numpy=1464.6365>, <tf.Tensor: id=70052, shape=(), dtype=float32, numpy=1462.1602>, <tf.Tensor: id=70497, shape=(), dtype=float32, numpy=1458.745>, <tf.Tensor: id=70942, shape=(), dtype=float32, numpy=1455.2542>]


The loss is pretty high but it does decrease anyway. The purpose of this notebook is to show how we can combine the low level TF ops with the high-level APIs to aid utmost flexibility in model definition. The example is inspired from [this tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html). 